# Project: **[Surprise](https://github.com/NicolasHug/Surprise)**

[Latest commit as of writing (on master)](https://github.com/NicolasHug/Surprise/commit/5eb9017bcbe19e16ae251b2598eb8b768368023b)

Surprise is a Python library maintained by one of the core members of sklearn (popular Python ML library), Nicolas Hub, for recommender systems. Recommender systems often use matrix factorization to find latent features that can relate a specific user to a set of items. One common example of an application is Netflix, where subscribers get recommended movies based on not only what they've already watched buy also what people with similar consumption habits watch.

Currently the repo has 3.9k stars and uses Travis CI for its [automated testing](https://travis-ci.org/github/NicolasHug/Surprise).

I was looking at a few of the top recommender systems on GitHub and I really liked this one because it uses [Cython](https://cython.org/). Cython is a compiler that adds some extra syntax on top of Python to make it compile to optimized C code. I've never used it before but the promise is that you can import Cython modules (.pyx extension) just as you would normal Python (.py) modules.

## Why factorize?

The problem with recommender systems is that the user-item matrices are very sparse. Generally you have user rows ($n$) and item columns ($m$). One row vector would be that user's preferences towards items $0..m$. Since generally most users do not interact with all items in a system (i.e. no one buys everything on Amazon or watches all Netflix movies), the resulting $n \times m$ matrix is often impractical.

The idea is to create some $k$ number of new features that we can use in estimating a factorization of the sparse $n \times m$ matrix. If we factorize, we end up with 2 smaller matrices, $n \times k$ and $k \times m$ that will multiply to approximate the full $n \times m$ matrix.

## Method: SVD

Several methods for factorizing in recommender systems are used in practice: SVD (singular value decomposition), SVD++, and NMF (non-negative matrix factorization). Those are all implemented by this library.

I will be looking at the implementation of SVD, since it was famously invented(?) by Simon Funk to win the [Netflix Prize](https://en.wikipedia.org/wiki/Netflix_Prize) back in 2009. Check out Funk's [blog post](https://sifter.org/~simon/journal/20061211.html) describing it.

[Click here to read the docs for the `SVD` class in Surprise.](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD)

The `SVD` class extends `AlgoBase` which has a few methods, but `fit` and `predict` are the most common. This is mirrors the API for sklearn, unsurprisingly.

The singular value decomposition is calculated using stochastic gradient descent (SGD).

### Questions I have about the method

## Example usage (movielens dataset)

## Open questions